# User Rep Scores
- Goal: output a csv for each user in the datahunt, with each row corresponding to a single answer
- only looking at the parent questions for now


In [1]:
import numpy as np
import pandas as pd
# import scipy as sp
# from scipy import stats

The file path of the datahunt is the only thing that needs to be inputted. This code assumes that the datahunt file name will contain the type of task in some form.

In [2]:
e_file = "testing-format/BETA_Evidence-2020-05-20T0111-DataHunt.csv"

l_file = "testing-format/BETA_Language-2020-05-20T0110-DataHunt.csv"

p_file = "testing-format/BETA_Probability-2020-05-20T0111-DataHunt.csv"

r_file = "testing-format/BETA_Reasoning-2020-05-20T0111-DataHunt.csv"

In [3]:
# fill this in
datahunt_file = l_file

In [4]:
# Kai's convergence answer csv
answers_df = pd.read_csv("convergence/Answer_Consensus.csv")

# read csv into df
df = pd.read_csv(datahunt_file)


In [5]:
# assigns a task type based on the name of the inputted datahunt csv
if "language" in datahunt_file.lower():
    task_type = "language"
elif "probability" in datahunt_file.lower():
    task_type = "probability"
elif "reasoning" in datahunt_file.lower():
    task_type = "reasoning"
else:
    task_type = "evidence"
    
print("Task type is " + task_type)

Task type is language


In [6]:
def get_question_base_label(row):
    if "A" not in row['Question Label']:
        return row['Question Label']
    else:
        base_end_index = row['Question Label'].index("A") - 1
        return row['Question Label'][:base_end_index]

In [7]:
# uses the function above to add the row "Question Base Label" to the answers df, so there is a uniform 
# format of question labels (select all questions were in a different form)
answers_df["Question Base Label"] = answers_df.apply(get_question_base_label, axis=1)
answers_df

,Unnamed: 0,Task File,Article Number,Question Label,Answer Label,Question Base Label
0,0,Evidence,1712,T1.Q1,T1.Q1.A1,T1.Q1
1,1,Evidence,1712,T1.Q10,T1.Q10.A5,T1.Q10
2,2,Evidence,1712,T1.Q11,T1.Q11.A2,T1.Q11
3,3,Evidence,1712,T1.Q12,T1.Q12.A3,T1.Q12
4,4,Evidence,1712,T1.Q13,T1.Q13.A5,T1.Q13
...,...,...,...,...,...,...
283,283,Probability,100026,T1.Q12.A2,1,T1.Q12
284,284,Reasoning,1712,T1.Q10,T1.Q10.A8,T1.Q10
285,285,Reasoning,1712,T1.Q9,T1.Q9.A5,T1.Q9
286,286,Reasoning,1712,T1.Q1.A2,1,T1.Q1


In [8]:
# array of questions that are "parent" questions
language_parents = ["T1.Q1", "T1.Q12"]
probability_parents = ["T1.Q1", "T1.Q5", "T1.Q6", "T1.Q11"]
reasoning_parents = ["T1.Q1"]
evidence_parents = ["T1.Q1", "T1.Q12"]

# corresponding list of question types, language_parents[n] maps to language_parent_types[n]
language_parents_types = ["select_all", "ordinal"]
probability_parents_types = ["ordinal", "ordinal", "select_one", "ordinal"]
reasoning_parents_types = ["select_all"]
evidence_parents_types = ["select_one", "ordinal"]

# corresponding list of max number of problems, used for scoring ordinal questions
language_num_answers = [13, 4]
probability_num_answers = [3, 3, 3, 4]
reasoning_num_answers = [6]
evidence_num_answers = [3, 4]

In [9]:
# cuts down answers_df to the rows with the relevant task type
if task_type == "language":
    task_answers_df = answers_df.loc[answers_df["Task File"] == "Language"]
    parent_questions = language_parents
    num_answers = language_num_answers
elif task_type == "probability":
    task_answers_df = answers_df.loc[answers_df["Task File"] == "Probability"]
    parent_questions = probability_parents
    num_answers = probability_num_answers
elif task_type == "reasoning":
    task_answers_df = answers_df.loc[answers_df["Task File"] == "Reasoning"]
    parent_questions = reasoning_parents
    num_answers = reasoning_num_answers
elif task_type == "evidence":
    task_answers_df = answers_df.loc[answers_df["Task File"] == "Evidence"]
    parent_questions = evidence_parents
    num_answers = evidence_num_answers
else:
    print("Invalid task type")


In [10]:
# set parent_df to the relevant columns of the parent questions
df = df[["contributor_uuid", "quiz_task_uuid", "article_number", "question_label", 
         "answer_label", "answer_text", "quiz_taskrun_uuid", "finish_time"]]

# creates a dictionary mapping the task type to an list of which questions are parent questions
task_parents = {"language": language_parents, "probability": probability_parents,
                "reasoning": reasoning_parents, "evidence": evidence_parents}
task_parents_types = {"language": language_parents_types, "probability": probability_parents_types,
                "reasoning": reasoning_parents_types, "evidence": evidence_parents_types}


parent_df = df.loc[df['question_label'].isin(task_parents[task_type])]

In [11]:
# adds a column with the question_type of the question_label
question_types = []
for i in parent_df['question_label']:
    question_type = task_parents_types[task_type][task_parents[task_type].index(i)]
    question_types.append(question_type)
        
parent_df.insert(2, "question_type", question_types, True)


In [12]:
# task_answers_df.loc[task_answers_df["Question Base Label"] == 'T1.Q1']
task_answers_df
task_answers_df.loc[task_answers_df["Question Base Label"] == "T1.Q12"].loc[task_answers_df["Article Number"] == 1712]

,Unnamed: 0,Task File,Article Number,Question Label,Answer Label,Question Base Label
72,72,Language,1712,T1.Q12,T1.Q12.A1,T1.Q12


Functions for each question type that take in a row from parent_df and output a row with the relevant information (including score) for that user.

Ordinal questions are scored by how far the answer was from the converged answer, where the penalty for each "one" they are off by is 1 / half the total number of questions.

Select-one questions are given scores of 1 if they select the converged answer, 0 otherwise.

Select-all questions are scored by finding the proportion of "correct" selections they selected, where a "correct" selection is 0 or 1, depending on if the converged set of selections for that question contained a given selection. Thus, false positives and false negatives are included in that calculation.

In [13]:
def ordinal_question(row):
    article_number = row["article_number"]
    question_label = row["question_label"]
    answer_label = int(row["answer_label"][-1])
    correct_answer = task_answers_df.loc[task_answers_df["Article Number"] == article_number]
    correct_answer = correct_answer.loc[correct_answer["Question Base Label"] == question_label]
    correct_answer = int(correct_answer["Answer Label"].iloc[0][-1])
    
    question_num_answers = num_answers[parent_questions.index(row["question_label"])]
    penalty = 1 / (question_num_answers / 2)
    
    return 1 - abs(answer_label - correct_answer) * penalty
    

In [14]:
def select_one_question(row):
    article_number = row["article_number"]
    question_label = row["question_label"]
    answer_label = row["answer_label"]
    correct_answer = task_answers_df.loc[task_answers_df["Article Number"] == article_number]
    correct_answer = correct_answer.loc[correct_answer["Question Base Label"] == question_label]
    correct_answer = correct_answer["Answer Label"].iloc[0]
    
    if answer_label == correct_answer:
        return 1
    else:
        return 0
    

In [15]:
def select_all_question(row):
    user_id = row["contributor_uuid"]
    response_id = row["quiz_taskrun_uuid"]
    
    if user_dfs[user_id].loc[user_dfs[user_id]["quiz_taskrun_uuid"] == response_id].empty:
        article_number = row["article_number"]
        question_label = row["question_label"]
        answer_label = row["answer_label"]

        question_user_df = parent_df.loc[parent_df["quiz_taskrun_uuid"] == response_id]
        question_answers_df = task_answers_df.loc[(task_answers_df["Article Number"] == article_number) &\
                                                 (task_answers_df["Question Base Label"] == question_label)]

        correct_question_counter = 0
        total_question_counter = 0
        
        for index, row in question_answers_df.iterrows():
            if question_user_df.loc[question_user_df["answer_label"] == row["Question Label"]].empty:
                if row["Answer Label"] == "0":
                    correct_question_counter += 1
            else:
                if row["Answer Label"] == "1":
                    correct_question_counter += 1
            total_question_counter += 1

        overall_question_score = correct_question_counter / total_question_counter

        return overall_question_score
    
    return -1
      

The below cell creates a library that maps contributor_uuids to a df where each row contains a question they answered and the score they received for that question

In [28]:
user_dfs = {}

for index, row in parent_df.iterrows():
    if row['contributor_uuid'] not in user_dfs:
        user_dfs[row['contributor_uuid']] = pd.DataFrame(columns=['quiz_task_uuid', 'quiz_taskrun_uuid', 'question_label', 'question_type', 'answer_score', 'time_stamp'])
    if row['question_type'] == "select_all":
        score = select_all_question(row)
    elif row['question_type'] == "select_one":
        score = select_one_question(row)
    else:
        score = ordinal_question(row)
    if score != -1:
        user_dfs[row['contributor_uuid']] = user_dfs[row['contributor_uuid']].append({'quiz_task_uuid': row['quiz_task_uuid'], 'quiz_taskrun_uuid': row['quiz_taskrun_uuid'], 'question_label': row['question_label'], 'task_type': task_type, 'question_type': row['question_type'], 'answer_score': score, 'time_stamp': row['finish_time']}, ignore_index=True)
        
    

In [29]:
user_dfs

{'f9143626-bfe0-4e69-b652-6d1525ab4eb0':                           quiz_task_uuid  \
 0   a25ed7d6-4f1b-40a4-9f72-54e13e5658eb   
 1   a25ed7d6-4f1b-40a4-9f72-54e13e5658eb   
 2   a25ed7d6-4f1b-40a4-9f72-54e13e5658eb   
 3   a25ed7d6-4f1b-40a4-9f72-54e13e5658eb   
 4   a25ed7d6-4f1b-40a4-9f72-54e13e5658eb   
 5   a25ed7d6-4f1b-40a4-9f72-54e13e5658eb   
 6   f65e3f32-7347-42b3-a6b2-39bc62e226c2   
 7   f65e3f32-7347-42b3-a6b2-39bc62e226c2   
 8   f65e3f32-7347-42b3-a6b2-39bc62e226c2   
 9   f65e3f32-7347-42b3-a6b2-39bc62e226c2   
 10  f65e3f32-7347-42b3-a6b2-39bc62e226c2   
 11  f65e3f32-7347-42b3-a6b2-39bc62e226c2   
 12  8a155187-7903-4284-97a2-c40404008deb   
 13  8a155187-7903-4284-97a2-c40404008deb   
 14  8a155187-7903-4284-97a2-c40404008deb   
 15  8a155187-7903-4284-97a2-c40404008deb   
 16  8a155187-7903-4284-97a2-c40404008deb   
 17  8a155187-7903-4284-97a2-c40404008deb   
 18  946cc55d-ec61-41df-bf09-3359b9dc76e2   
 19  946cc55d-ec61-41df-bf09-3359b9dc76e2   
 20  946cc55d-e

In [30]:
# example of df for one user.
user_dfs["0e4b8a3e-2714-4b5f-ba10-bb576aaa5cc6"]

,quiz_task_uuid,quiz_taskrun_uuid,question_label,question_type,answer_score,time_stamp,task_type
0,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,7b02602a-9c3c-4de8-a93d-d15a3aa477e6,T1.Q1,select_all,0.750000,2019-09-16 23:25:37.329830,language
1,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,7b02602a-9c3c-4de8-a93d-d15a3aa477e6,T1.Q12,ordinal,1.000000,2019-09-16 23:25:37.329830,language
2,f65e3f32-7347-42b3-a6b2-39bc62e226c2,dc0db990-79bd-49fb-ab21-549d514fe81e,T1.Q1,select_all,0.750000,2019-09-16 23:36:01.249258,language
3,f65e3f32-7347-42b3-a6b2-39bc62e226c2,dc0db990-79bd-49fb-ab21-549d514fe81e,T1.Q12,ordinal,1.000000,2019-09-16 23:36:01.249258,language
4,8a155187-7903-4284-97a2-c40404008deb,cc85610d-ec4d-4c7c-afe4-9108ef5b2d08,T1.Q1,select_all,0.769231,2019-09-16 23:41:43.882877,language
5,8a155187-7903-4284-97a2-c40404008deb,cc85610d-ec4d-4c7c-afe4-9108ef5b2d08,T1.Q12,ordinal,0.500000,2019-09-16 23:41:43.882877,language
6,8a155187-7903-4284-97a2-c40404008deb,b1cd7899-9e04-43f2-9ac3-9438a7baa3ea,T1.Q1,select_all,0.769231,2019-09-16 23:41:43.929460,language
7,8a155187-7903-4284-97a2-c40404008deb,b1cd7899-9e04-43f2-9ac3-9438a7baa3ea,T1.Q12,ordinal,0.500000,2019-09-16 23:41:43.929460,language
8,946cc55d-ec61-41df-bf09-3359b9dc76e2,7da41488-d6a0-4e8d-9c92-dc38190d6c60,T1.Q1,select_all,0.923077,2019-09-16 23:48:27.897041,language
9,946cc55d-ec61-41df-bf09-3359b9dc76e2,7da41488-d6a0-4e8d-9c92-dc38190d6c60,T1.Q12,ordinal,0.000000,2019-09-16 23:48:27.897041,language
